# COVID-19: United States Confirmed Cases & Deaths 

The following data cleaning and merging is performed to load the data into Power BI for further visualization and analysis.

In [1]:
# Packages / libraries
import os 
import numpy as np 
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

The data collected for this analysis is operated by the Johns Hopkins University Center for Systems Science and Engineering (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [2]:
# load raw data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_deaths_US.csv")

# Confirm that the dataframes have the same number of rows (countries) and columns (dates)
# Deaths dataframe has an extra column 'Population'
print('The shape of confirmed is:', confirmed.shape)
print('The shape of deaths is:', deaths.shape)

confirmed.head()
#deaths.head()

The shape of confirmed is: (3340, 193)
The shape of deaths is: (3340, 194)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,310,312,312,313,314,314,314,314,319,319
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,33,33,36,36,37,37,37,37,38,38
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,...,63,66,66,66,66,67,68,72,72,72
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,...,54,57,58,60,60,66,67,70,71,75


In [3]:
# Drop unnecessary columns
confirmed.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)
deaths.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)

#confirmed.head()
#deaths.head()

In [4]:
# melt the date columns into one column
confirmed2 = pd.melt(confirmed, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_'], var_name=['Date'])
deaths2 = pd.melt(deaths, id_vars= ['UID','Admin2','Province_State','Country_Region','Lat','Long_','Population'], var_name=['Date'])

print('The shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)

confirmed2.head()

The shape of confirmed is: (607880, 8)
The shape of deaths is: (607880, 9)


,UID,Admin2,Province_State,Country_Region,Lat,Long_,Date,value
0,16,NaN,American Samoa,US,-14.271000,-170.132000,1/22/20,0
1,316,NaN,Guam,US,13.444300,144.793700,1/22/20,0
2,580,NaN,Northern Mariana Islands,US,15.097900,145.673900,1/22/20,0
3,63072001,Adjuntas,Puerto Rico,US,18.180117,-66.754367,1/22/20,0
4,63072003,Aguada,Puerto Rico,US,18.360255,-67.175131,1/22/20,0


In [5]:
# convert Date column into datetime objects (xxxx-xx-xx)
confirmed2['Date'] = pd.to_datetime(confirmed2['Date'])
deaths2['Date'] = pd.to_datetime(deaths2['Date'])

confirmed2.head()

,UID,Admin2,Province_State,Country_Region,Lat,Long_,Date,value
0,16,NaN,American Samoa,US,-14.271000,-170.132000,2020-01-22,0
1,316,NaN,Guam,US,13.444300,144.793700,2020-01-22,0
2,580,NaN,Northern Mariana Islands,US,15.097900,145.673900,2020-01-22,0
3,63072001,Adjuntas,Puerto Rico,US,18.180117,-66.754367,2020-01-22,0
4,63072003,Aguada,Puerto Rico,US,18.360255,-67.175131,2020-01-22,0


In [6]:
# Replace column headings with detailed fields
confirmed2.columns = confirmed2.columns.str.replace('value','Confirmed')
confirmed2.columns = confirmed2.columns.str.replace('Admin2','County')
deaths2.columns = deaths2.columns.str.replace('value', 'Deaths')
deaths2.columns = deaths2.columns.str.replace('Admin2','County')

# Investigate NULLs before join
print(confirmed2.isnull().sum())
print(deaths2.isnull().sum())

UID                  0
County            1092
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Date                 0
Confirmed            0
dtype: int64
UID                  0
County            1092
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Population           0
Date                 0
Deaths               0
dtype: int64


The null county values are a result of the District of Columbia, and the 5 inhabited US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, US Virgin Islands) not having counties, in addition to the Grand Princess and Diamond Princess cruise ships. The null county value will be filled with the 'Province_State' name of the corresponding territory or cruise ship. 

In [7]:
# Join the dataframes 
covid = confirmed2.merge(deaths2[['UID','Province_State','Country_Region','Date','Deaths','Population']],
                      how='outer',
                      left_on=['UID','Province_State','Country_Region','Date'],
                      right_on=['UID','Province_State','Country_Region','Date'])

# Investigate the shape of the dataframe after the join
print('\nThe shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)
print('The shape of the joined dataframe is:', covid.shape)

#print(covid.isnull().sum())
#covid.tail()


The shape of confirmed is: (607880, 8)
The shape of deaths is: (607880, 9)
The shape of the joined dataframe is: (607880, 10)


The joined dataframe "covid" has 10 columns because Deaths and Population has been added to the confirmed dataframe. 

In [8]:
# Fill County NaN values with Province_State values (Applies only to DC, US territories, and cruise ships)
covid['County'].fillna(covid['Province_State'],inplace=True)

#covid.head()
covid.isnull().sum()

UID               0
County            0
Province_State    0
Country_Region    0
Lat               0
Long_             0
Date              0
Confirmed         0
Deaths            0
Population        0
dtype: int64

In [9]:
# Add Month-Year column
covid['Month-Year'] = covid['Date'].dt.strftime('%b-%Y')
#covid.tail()


In [10]:
# Copy df
temp = covid.copy()

# Create columns for previous date's COVID-19 cases to create daily numbers and a running total 
temp['Temp Date'] = covid['Date'] + pd.Timedelta(days=1)
temp.rename(columns={'Confirmed':'Confirmed - 1', 'Deaths':'Deaths - 1', 'Date':'Date - 1'}, inplace=True)

# Perform left join on DFs
covid_19 = covid.merge(temp[['UID','Province_State','Country_Region','Confirmed - 1','Deaths - 1',
                             'Temp Date','Date - 1']], how='left', 
                       left_on=['UID','Province_State','Country_Region','Date'],
                       right_on=['UID','Province_State','Country_Region','Temp Date'])
print(covid_19.shape)
covid_19.head()

(607880, 15)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1
0,16,American Samoa,American Samoa,US,-14.271000,-170.132000,2020-01-22,0,0,55641,Jan-2020,NaN,NaN,NaT,NaT
1,316,Guam,Guam,US,13.444300,144.793700,2020-01-22,0,0,164229,Jan-2020,NaN,NaN,NaT,NaT
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.097900,145.673900,2020-01-22,0,0,55144,Jan-2020,NaN,NaN,NaT,NaT
3,63072001,Adjuntas,Puerto Rico,US,18.180117,-66.754367,2020-01-22,0,0,19483,Jan-2020,NaN,NaN,NaT,NaT
4,63072003,Aguada,Puerto Rico,US,18.360255,-67.175131,2020-01-22,0,0,41959,Jan-2020,NaN,NaN,NaT,NaT


In [11]:
# Calculate the daily numbers for confirmed cases & deaths (current aggregate - previous aggregate = daily confirmed)
covid_19['Daily Confirmed'] = covid_19['Confirmed'] - covid_19['Confirmed - 1']
covid_19['Daily Deaths'] = covid_19['Deaths'] - covid_19['Deaths - 1']

print(covid_19.shape)
covid_19.head()

(607880, 17)


,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Confirmed - 1,Deaths - 1,Temp Date,Date - 1,Daily Confirmed,Daily Deaths
0,16,American Samoa,American Samoa,US,-14.271000,-170.132000,2020-01-22,0,0,55641,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
1,316,Guam,Guam,US,13.444300,144.793700,2020-01-22,0,0,164229,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.097900,145.673900,2020-01-22,0,0,55144,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
3,63072001,Adjuntas,Puerto Rico,US,18.180117,-66.754367,2020-01-22,0,0,19483,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN
4,63072003,Aguada,Puerto Rico,US,18.360255,-67.175131,2020-01-22,0,0,41959,Jan-2020,NaN,NaN,NaT,NaT,NaN,NaN


In [12]:
# Include daily numbers for the first day of data where there is no previous date available (2020-01-22)
covid_19['Daily Confirmed'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Confirmed']
covid_19['Daily Deaths'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Deaths']

# Delete unnecessary columns
del covid_19['Confirmed - 1'] 
del covid_19['Deaths - 1']
del covid_19['Temp Date']
del covid_19['Date - 1']

covid_19.head()

,UID,County,Province_State,Country_Region,Lat,Long_,Date,Confirmed,Deaths,Population,Month-Year,Daily Confirmed,Daily Deaths
0,16,American Samoa,American Samoa,US,-14.271000,-170.132000,2020-01-22,0,0,55641,Jan-2020,0.0,0.0
1,316,Guam,Guam,US,13.444300,144.793700,2020-01-22,0,0,164229,Jan-2020,0.0,0.0
2,580,Northern Mariana Islands,Northern Mariana Islands,US,15.097900,145.673900,2020-01-22,0,0,55144,Jan-2020,0.0,0.0
3,63072001,Adjuntas,Puerto Rico,US,18.180117,-66.754367,2020-01-22,0,0,19483,Jan-2020,0.0,0.0
4,63072003,Aguada,Puerto Rico,US,18.360255,-67.175131,2020-01-22,0,0,41959,Jan-2020,0.0,0.0


In [13]:
# calculate weekly sum by state
weekly_covid = covid_19.groupby(['Province_State', covid_19['Date'].dt.strftime('%W')]).agg({'Daily Confirmed':'sum','Daily Deaths':'sum'}).reset_index()

# replace 'Date' with 'Week Number' of the year 
weekly_covid.columns = weekly_covid.columns.str.replace('Date','Week Number')
weekly_covid.columns = weekly_covid.columns.str.replace('Daily Confirmed','Weekly Confirmed')
weekly_covid.columns = weekly_covid.columns.str.replace('Daily Deaths','Weekly Deaths')

# view weekly numbers by state
weekly_covid[weekly_covid['Week Number'] == weekly_covid['Week Number'].max()]

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths
26,Alabama,29,3347.0,16.0
53,Alaska,29,167.0,0.0
80,American Samoa,29,0.0,0.0
107,Arizona,29,5059.0,157.0
134,Arkansas,29,1427.0,17.0
161,California,29,22399.0,173.0
188,Colorado,29,917.0,11.0
215,Connecticut,29,203.0,10.0
242,Delaware,29,227.0,2.0
269,Diamond Princess,29,0.0,0.0


In [14]:
# calculate the weekly percentage change in confirmed cases and deaths for each state
weekly_change = weekly_covid.groupby(['Province_State', 'Week Number']).agg({'Weekly Confirmed':'sum','Weekly Deaths':'sum'}).pct_change().reset_index()
weekly_change

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths
0,Alabama,03,NaN,NaN
1,Alabama,04,NaN,NaN
2,Alabama,05,NaN,NaN
3,Alabama,06,NaN,NaN
4,Alabama,07,NaN,NaN
...,...,...,...,...
1561,Wyoming,25,0.605839,-1.0
1562,Wyoming,26,-0.013636,NaN
1563,Wyoming,27,0.050691,inf
1564,Wyoming,28,0.157895,3.0


In [15]:
# add the weekly percentage change calculated above to the weekly_covid dataframe
weekly_covid[['Weekly Confirmed % Change', 'Weekly Deaths % Change']] = weekly_change[['Weekly Confirmed','Weekly Deaths']]

# replace NaN with 0
weekly_covid.replace(np.nan, 0, inplace=True)

weekly_covid

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths,Weekly Confirmed % Change,Weekly Deaths % Change
0,Alabama,03,0.0,0.0,0.000000,0.0
1,Alabama,04,0.0,0.0,0.000000,0.0
2,Alabama,05,0.0,0.0,0.000000,0.0
3,Alabama,06,0.0,0.0,0.000000,0.0
4,Alabama,07,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...
1561,Wyoming,25,220.0,0.0,0.605839,-1.0
1562,Wyoming,26,217.0,0.0,-0.013636,0.0
1563,Wyoming,27,228.0,1.0,0.050691,inf
1564,Wyoming,28,264.0,4.0,0.157895,3.0


In [16]:
# check the most recent daily confirmed cases
daily_sum = covid_19.groupby(['Date']).agg({'Daily Confirmed': 'sum'})
print(daily_sum.tail(10))

# check the most recent daily deaths 
daily_deaths = covid_19.groupby(['Date']).agg({'Daily Deaths': 'sum'})
print(daily_deaths.tail(10))

            Daily Confirmed
Date                       
2020-07-12          59017.0
2020-07-13          59215.0
2020-07-14          67417.0
2020-07-15          67328.0
2020-07-16          77255.0
2020-07-17          71558.0
2020-07-18          63698.0
2020-07-19          61847.0
2020-07-20          61417.0
2020-07-21          64534.0
            Daily Deaths
Date                    
2020-07-12         428.0
2020-07-13         361.0
2020-07-14         900.0
2020-07-15         949.0
2020-07-16         943.0
2020-07-17         908.0
2020-07-18         853.0
2020-07-19         415.0
2020-07-20         379.0
2020-07-21        1082.0


In [17]:
# Export Data as csv to load into Power BI
covid_19.to_csv('UScovid', sep='\t')
weekly_covid.to_csv('weekly_covid', sep='\t')